Based on a tutorial at

https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/


In [1]:
import pandas as pd
import numpy as np

“use_id” is shared between the user_usage and user_device

probably: 
* for every user_id there are N use_ids

“device” column of user_device and “Model” column of the devices dataset contain common codes

In [5]:
user_usage = pd.read_csv("https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/user_usage.csv", sep=',')
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [6]:
user_device = pd.read_csv("https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/user_device.csv")
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [13]:
android_devices = pd.read_csv("https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/android_devices.csv")

In [14]:
android_devices.head()

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A


---
We would like to determine if the usage patterns for users differ between different devices. For example, do users using Samsung devices use more call minutes than those using  LG devices? This is a toy problem given the small sample size in these dataset, but is a perfect example of where merges are required.

need to merge user_usage with model information

Test the assertion that there for every use_id in A there is the use_id in B 

In [21]:
len( user_usage.use_id.unique() )

240

In [22]:
len(user_usage)

240

So use_id is a unique key in user_usage.
Is it also unique in user_device?

In [26]:
user_device.use_id.value_counts().max()

1

Yes, it is, because every value appears at most once there.

So is there a match of use_ids between the two tables for every value of use_id on either side?
If we do inner join on use_id and it has as many rows as the input tables, it means there's no use_id on either side for which there wouldn't be a match on the other side. Is this true? What if there are 50 use_ids on the left, 50 use_ids on the right, 5 are on the left with no match on the right. Then there would be only 45 rows in the result. So yeah, that should work.

In [28]:
m1 = pd.merge(left=user_usage, right=user_device, how='inner', on='use_id')

In [30]:
len(user_usage), len(user_device), len(m1)

(240, 272, 159)

OK, so there are differences in use_id. There are only 159 matches. Which ones are they?

use_ids on the left with no match on the right:

In [49]:
m_outer_join = pd.merge(left=user_usage, right=user_device, how='outer', on='use_id', indicator=True)
m_outer_join[m_outer_join._merge=='left_only'][['use_id', '_merge']].count()

use_id    81
_merge    81
dtype: int64

In [51]:
m_outer_join[m_outer_join._merge=='right_only'][['use_id', '_merge']].count()

use_id    113
_merge    113
dtype: int64

In [53]:
m_outer_join[m_outer_join._merge=='both'][['use_id', '_merge']].count()

use_id    159
_merge    159
dtype: int64

In [58]:
81+113+159 # left_only + right_only + both

353

In [59]:
240+272-159 # left + right - both

353

So, there are use_ids on the left with no match on the right as well as use_ids on the right with no match on the left. 

Assertion: there is a 1:N relationship in table user_device between user_id and use_id. For each use_id there is exactly one user_id. 
How do we verify this assertion?
We might look at multiindex, but that's just for illustration:

In [70]:
user_device.set_index(['user_id', 'use_id']).head(12)

platform  platform_version     device  use_type_id
user_id use_id                                                   
26980   22782       ios              10.2  iPhone7,2            2
29628   22783   android               6.0    Nexus 5            3
28473   22784   android               5.1   SM-G903F            1
15200   22785       ios              10.2  iPhone7,2            3
28239   22786   android               6.0  ONE E1003            1
12921   22787   android               4.3   GT-I9505            1
28714   22788   android               6.0   SM-G930F            1
        22789   android               6.0   SM-G930F            1
29592   22790   android               5.1      D2303            1
28775   22791       ios              10.2  iPhone6,2            3
28217   22792   android               5.1   SM-G361F            1
        22793   android               5.1   SM-G361F            1

Finding rows in a Pandas DataFrame with columns that violate a one-to-one mapping

https://stackoverflow.com/questions/24005064/finding-rows-in-a-pandas-dataframe-with-columns-that-violate-a-one-to-one-mappin

In [88]:
g = user_device[['use_id', 'user_id']].groupby(by = ['user_id'])
a = g.transform(lambda x: len(x.unique()))

I don't understand this.

Let's do the merge

In [119]:
m = pd.merge(left=user_usage, right=user_device, how = 'left', on='use_id', indicator=True)

In [103]:
for i, r in user_usage.iterrows():
    if not (user_usage.index[i]==m.index[i]):
        print('not equal:', i)

In [106]:
m

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,user_id,platform,platform_version,device,use_type_id,_merge
0,21.97,4.82,1557.33,22787,12921.0,android,4.3,GT-I9505,1.0,both
1,1710.08,136.88,7267.55,22788,28714.0,android,6.0,SM-G930F,1.0,both
2,1710.08,136.88,7267.55,22789,28714.0,android,6.0,SM-G930F,1.0,both
3,94.46,35.17,519.12,22790,29592.0,android,5.1,D2303,1.0,both
4,71.59,79.26,1557.33,22792,28217.0,android,5.1,SM-G361F,1.0,both
5,71.59,79.26,1557.33,22793,28217.0,android,5.1,SM-G361F,1.0,both
6,71.59,79.26,519.12,22794,28217.0,android,5.1,SM-G361F,1.0,both
7,71.59,79.26,519.12,22795,28217.0,android,5.1,SM-G361F,1.0,both
8,30.92,22.77,3114.67,22799,29643.0,android,6.0,ONEPLUS A3003,1.0,both
9,69.80,14.70,25955.55,22801,10976.0,android,4.4,GT-I9505,1.0,both


This is a fantastic command

In [114]:
m_outer_join[m_outer_join._merge=='both'][['use_id', '_merge']].count()

use_id    159
_merge    159
dtype: int64

This one is how many values are in common. 
It's also the length of the inner join.

In [115]:
user_usage['use_id'].isin(user_device['use_id']).value_counts()

True     159
False     81
Name: use_id, dtype: int64

In [116]:
user_device['use_id'].isin(user_usage['use_id']).value_counts()

True     159
False    113
Name: use_id, dtype: int64

In [118]:
m.columns

Index(['outgoing_mins_per_month', 'outgoing_sms_per_month', 'monthly_mb',
       'use_id', 'user_id', 'platform', 'platform_version', 'device',
       'use_type_id', '_merge'],
      dtype='object')

In [128]:
m2 = pd.merge(left = m, right = android_devices[['Retail Branding', 'Model']], 
              left_on='device', right_on = 'Model',
             how = 'left')
m2.sort_values(by='use_id', ascending=True, inplace=True)

In [143]:
m2.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,user_id,platform,platform_version,device,use_type_id,_merge,manufacturer,Model
0,21.97,4.82,1557.33,22787,12921.0,android,4.3,GT-I9505,1.0,both,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,28714.0,android,6.0,SM-G930F,1.0,both,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,28714.0,android,6.0,SM-G930F,1.0,both,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,29592.0,android,5.1,D2303,1.0,both,Sony,D2303
4,71.59,79.26,1557.33,22792,28217.0,android,5.1,SM-G361F,1.0,both,Samsung,SM-G361F


In [135]:
m2.rename(columns={'Retail Branding':'manufacturer'}, inplace=True)

In [136]:
m2.columns

Index(['outgoing_mins_per_month', 'outgoing_sms_per_month', 'monthly_mb',
       'use_id', 'user_id', 'platform', 'platform_version', 'device',
       'use_type_id', '_merge', 'manufacturer', 'Model'],
      dtype='object')

In [138]:
m3 = m2[['outgoing_mins_per_month', 'outgoing_sms_per_month', 'monthly_mb',
       'use_id', 'platform', 'device', 'manufacturer', 'Model']]
m3.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer,Model
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303,Sony,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung,SM-G361F


In [142]:
m3.groupby('manufacturer').agg({
    'outgoing_mins_per_month':"mean",
    'outgoing_sms_per_month':"mean",
    'monthly_mb':'mean',
    'use_id':"count"
})

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
manufacturer,,,,
HTC,299.842955,93.059318,5144.077955,44
Huawei,81.526667,9.500000,1561.226667,3
LGE,111.530000,12.760000,1557.330000,2
Lava,60.650000,261.900000,12458.670000,2
Lenovo,215.920000,12.930000,1557.330000,2
Motorola,95.127500,65.666250,3946.500000,16
OnePlus,354.855000,48.330000,6575.410000,6
Samsung,191.010093,92.390463,4017.318889,108
Sony,177.315625,40.176250,3212.000625,16
